In [2]:
import numpy as np 
import pandas as pd
import geopandas as gpd

In [ ]:
import os
os.getcwd()

In [ ]:
user_folder = <path to user_folder>

In [ ]:
gdf = gpd.read_file(rf'c:\Users\{user_folder}\Documents\rune_gdf.geojson')
gdf

,image_id,date,layer,geometry
0,0,2017-05-05 17:17:04,20170505t171704,"MULTIPOLYGON (((6.16768 57.08449, 6.16929 57.0..."
1,0,2017-05-05 17:17:04,20170505t171704,"MULTIPOLYGON (((6.17438 57.076, 6.17687 57.074..."
2,0,2017-05-05 17:17:04,20170505t171704,"MULTIPOLYGON (((6.20534 57.09317, 6.20732 57.0..."
3,1,2017-05-06 05:56:49,20170506t055649,"MULTIPOLYGON (((3.4011 56.38135, 3.39955 56.38..."
4,2,2017-05-07 05:47:47,20170507t054747,"MULTIPOLYGON (((6.16631 57.08426, 6.16849 57.0..."
...,...,...,...,...
455,112,2021-06-25 06:00:04,20210625t0604,"MULTIPOLYGON (((3.94577 57.03774, 3.94429 57.0..."
456,113,2021-06-25 06:04:42,20210625t060442,"MULTIPOLYGON (((3.70773 56.82461, 3.70709 56.8..."
457,113,2021-06-25 06:04:42,20210625t060442,"MULTIPOLYGON (((4.30786 56.67483, 4.30703 56.6..."
458,113,2021-06-25 06:04:42,20210625t060442,"MULTIPOLYGON (((4.14688 56.68461, 4.14582 56.6..."


In [ ]:
gdf_unique = gpd.read_file(rf'c:\Users\{user_folder}\Documents\rune_gdf_unique-image.geojson')

In [4]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

# ASF Sentinel-1 search and Download

In [ ]:
img_path = rf'C:\Users\{user_folder}\--Projects\Local\ArcPy\s1\32\measurement\s1b-iw-grd-vv-20170704t060417-20170704t060442-006334-00b22a-001.tiff'

In [1]:
import asf_search as asf

In [2]:
username = input('Enter ASF username:')

import getpass

password = getpass.getpass(prompt='Enter ASF password: ')
session = asf.ASFSession().auth_with_creds(username, password)

In [3]:
import tkinter as tk
from tkinter import filedialog

# Initialize the tkinter root window (it won't be shown)
root = tk.Tk()
root.withdraw()  # Hides the root window

# Open a directory selection dialog
root_path = filedialog.askdirectory(title="Select a root path")

In [4]:
from datetime import date 
print(date(2017,2,3))

2017-02-03


In [6]:
import pandas as pd

In [27]:
gdf_unique.geometry

0      MULTIPOLYGON (((6.16768 57.08449, 6.16929 57.0...
1      MULTIPOLYGON (((3.4011 56.38135, 3.39955 56.38...
2      MULTIPOLYGON (((6.16631 57.08426, 6.16849 57.0...
3      MULTIPOLYGON (((5.66545 56.99826, 5.66938 56.9...
4      MULTIPOLYGON (((5.66399 56.99815, 5.66768 56.9...
                             ...                        
110    MULTIPOLYGON (((4.20047 57.68632, 4.2018 57.68...
111    MULTIPOLYGON (((3.94316 57.03799, 3.94178 57.0...
112    MULTIPOLYGON (((3.94577 57.03774, 3.94429 57.0...
113    MULTIPOLYGON (((3.70773 56.82461, 3.70709 56.8...
114    MULTIPOLYGON (((3.94377 57.03823, 3.94516 57.0...
Name: geometry, Length: 115, dtype: geometry

In [29]:
from shapely.geometry import Point
geom = list(gdf_unique.loc[0].geometry.geoms[0].exterior.coords)[0]
geom = Point(geom).wkt
geom

'POINT (6.167679618132044 57.08449365258807)'

In [ ]:

# Query results from the ASF API
results = asf.search(platform=[asf.PLATFORM.SENTINEL1], 
                    processingLevel=[asf.PRODUCT_TYPE.GRD_HD],
                    intersectsWith=geom, 
                    start = date(2017,2,3),
                    # end = row['date'],
                    # start = row['date'] - pd.Timedelta(hours=1),
                    end = date(2020,2,3) + pd.Timedelta(days=40), # if we want one day more or less,
                    polarization='VV', # Keeps only the co-polarization band
                    # maxResults=3
                    )
print(len(results))
print(results)

0
{
  "features": [],
  "type": "FeatureCollection"
}


In [ ]:
from shapely.geometry import Point
import os 

for index, row in empty_gdf.iterrows():
    n = row["image_id"]
        # Define the full path for the download directory
    download_dir = os.path.join(root_path, str(n))
    
    # Ensure the directory exists
    os.makedirs(download_dir, exist_ok=True)
    
    # Access the 'geometry' column from the row
        # Access the 'geometry' column
    geometry = row['geometry']
    print(row['date'])
    
    # Check if the geometry is a Polygon or MultiPolygon
    if geometry.geom_type == 'Polygon':
        first_point = geometry.exterior.coords[0]  # Get the first point of the exterior
    elif geometry.geom_type == 'MultiPolygon':
        # For MultiPolygon, use the first polygon's exterior
        first_point = list(geometry.geoms[0].exterior.coords)[0]
    else:
        raise ValueError(f"Unsupported geometry type: {geometry.geom_type}")
    
    # Convert the first point to a Shapely Point object
    geom_wkt = Point(first_point).wkt

    # Query results from the ASF API
    results = asf.search(platform=[asf.PLATFORM.SENTINEL1], 
                        processingLevel=[asf.PRODUCT_TYPE.GRD_HD],
                        intersectsWith=geom_wkt, 
                        start = row['date'],
                        # end = row['date'],
                        # start = row['date'] - pd.Timedelta(hours=1),
                        end = row['date'] + pd.Timedelta(hours=20), # if we want one day more or less
                        maxResults=3)
    print(len(results))
    print(results)
    results.download(path=rf'{root_path}/{n}', session=session)
    print(f'processed {n} over {len(gdf_unique)} images')

In [386]:
def count_files_in_subfolders(root_folder):
    total_files = 0  # Initialize a counter for the total number of files

    # Walk through the root folder and its subfolders
    for dirpath, dirnames, filenames in os.walk(root_folder):
        # Add the number of files in the current directory to the total
        total_files += len(filenames)

    return total_files

# Count the total items in subfolders
total_items = count_files_in_subfolders(root_path)
print(f"Total number of items in subfolders: {total_items}")

Total number of items in subfolders: 119


In [382]:
import os

def find_empty_subfolders(root_folder):
    empty_subfolders = []  # List to store names of empty subfolders

    # Walk through the root folder and its subfolders
    for dirpath, dirnames, filenames in os.walk(root_folder):
        # Check if the directory is empty (no files or subfolders)
        if not dirnames and not filenames:
            folder_name = os.path.basename(dirpath)  # Get only the folder name
            empty_subfolders.append(folder_name)  # Add the folder name to the list

    return empty_subfolders

# Call the function and print the results
empty_folders = find_empty_subfolders(root_path)
if empty_folders:
    print("Empty subfolders:")
    for folder in empty_folders:
        print(folder)
else:
    print("No empty subfolders found.")

No empty subfolders found.


In [380]:
# Ensure the empty_folders list contains integers
empty_folders = [int(folder) for folder in empty_folders]

# Filter rows based on the empty_folders list
empty_gdf = gdf_unique.loc[gdf_unique.index.isin(empty_folders)]
empty_gdf

,image_id,date,layer,geometry
53,53,2020-04-30,slicks_20200430,"MULTIPOLYGON (((4.2741 56.64316, 4.27653 56.64..."
56,56,2020-05-02,slicks_20200502,"MULTIPOLYGON (((3.81586 56.64153, 3.81871 56.6..."
57,57,2020-05-05,slicks_20200505,"MULTIPOLYGON (((3.81898 56.64303, 3.8183 56.64..."
58,58,2020-05-06,slicks_20200506,"MULTIPOLYGON (((3.8199 56.64279, 3.81947 56.64..."
60,60,2020-05-07,slicks_20200507,"MULTIPOLYGON (((3.81913 56.64279, 3.82044 56.6..."
61,61,2020-05-08,slicks_20200508,"MULTIPOLYGON (((4.42516 56.65122, 4.42381 56.6..."
63,63,2020-05-11,slicks_20200511,"MULTIPOLYGON (((4.24214 56.59172, 4.24265 56.5..."
64,64,2020-05-12,slicks_20200512,"MULTIPOLYGON (((4.23636 56.5944, 4.23915 56.59..."
67,67,2020-05-14,slicks_20200514,"MULTIPOLYGON (((4.23643 56.59348, 4.23857 56.5..."
68,68,2020-05-17,slicks_20200517,"MULTIPOLYGON (((4.23696 56.59211, 4.23596 56.5..."


In [ ]:
import os
import zipfile

# Function to unzip files in tier 1 subfolders
def unzip_files_in_directory(root_dir):
    # Iterate through only the first level of subdirectories
    for foldername in os.listdir(root_dir):
        folder_path = os.path.join(root_dir, foldername)
        
        if os.path.isdir(folder_path):  # Check if it's a subfolder
            # Iterate through all files in this folder
            for filename in os.listdir(folder_path):
                if filename.endswith('.zip'):  # Check if the file is a .zip file
                    zip_path = os.path.join(folder_path, filename)
                    
                    # Extract the .zip file to the same directory
                    try:
                        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                            zip_ref.extractall(folder_path)  # Extract to the same folder
                        print(f"Successfully unzipped: {zip_path} to {folder_path}")
                        
                        # # Delete the .zip file after extraction
                        # os.remove(zip_path)
                        # print(f"Deleted zip file: {zip_path}")
                    except zipfile.BadZipFile:
                        print(f"Failed to unzip: {zip_path} (Bad Zip File)")

# Call the function to unzip files
unzip_files_in_directory(root_path)
